In [2]:
#django env setup
import os, sys
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "intelliwiz_config,settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [3]:
# Model imports
from apps.activity.models import Asset, Attachment, Device, DeviceEventlog, Event, Job, Jobneed, JobneedDetails, Location, Question, QuestionSet, QuestionSetBelonging, WorkPermit
from apps.attendance.models import PeopleEventlog, TestGeo, Tracking
from apps.onboarding.models import Bt, Contract, ContractDetail, GeofenceMaster, Shift, SitePeople, TypeAssist, WizardDraft
from apps.peoples.models import Capability, People, PermissionGroup, Pgbelonging, Pgroup
from apps.reminder.models import Reminder
from apps.tenants.models import Tenant
from apps.y_helpdesk.models import EscalationMatrix, Ticket
from apps.core import utils
import pandas as pd

# Shell Plus Django Imports
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, ExpressionWrapper, Value as V
from django.db.models.functions import Concat
from django.utils import timezone
from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery, FloatField, CharField
from datetime import datetime, timedelta

In [4]:
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets
from IPython.display import display as ipy_display

def dq(queryset):
    if not queryset: return None
    if queryset := list(queryset):
    
        if isinstance(queryset, (list, tuple)):
            # Handle values_list() queryset
            df = pd.DataFrame(queryset)
        elif isinstance(queryset, dict):
            # Handle values() queryset
            df = pd.DataFrame.from_records([queryset])
        else:
            # Handle regular queryset
            data = list(queryset.values())
            df = pd.DataFrame(data)

        # Set pandas display options to show all rows and columns
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)

        # Get the specified column names
        columns = list(queryset[0].keys())

        # Filter the DataFrame based on the specified columns
        df_filtered = df[columns]

        # Create a scrollable output widget
        output_widget = widgets.Output()

        # Display the DataFrame inside the output widget
        with output_widget:
            clear_output()
            display(df_filtered)

        # Display the output widget
        ipy_display(output_widget)
    else:
        print("Empty Queryset")

In [5]:
different_jobstatus = Jobneed.objects.distinct('jobstatus').filter(identifier = 'TASK').order_by('jobstatus').values_list('jobstatus', flat=True)

In [15]:
different_jobstatus

<QuerySet ['ASSIGNED', 'AUTOCLOSED', 'COMPLETED']>

In [24]:
d = {}
for status in different_jobstatus:
    d[status] = Jobneed.objects.filter(
        plandatetime__date__gte = '2023-05-28',
        plandatetime__date__lte = '2023-07-08',
        identifier = 'TASK',
        jobstatus=status
    ).count()
d

{'ASSIGNED': 55, 'AUTOCLOSED': 586, 'COMPLETED': 9}

In [27]:
import datetime
from django.db.models import Q

d = {}

# assuming different_jobstatus is a list of different job statuses
for status in different_jobstatus:
    d[status] = Jobneed.objects.filter(
        Q(plandatetime__gte=datetime.datetime(2023, 5, 28)) &
        Q(plandatetime__lte=datetime.datetime(2023, 7, 8)) &
        Q(identifier='TASK') &
        Q(jobstatus=status)
    ).count()

print(d)


/home/redmine/envs/youtility4/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Jobneed.plandatetime received a naive datetime (2023-05-28 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/redmine/envs/youtility4/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Jobneed.plandatetime received a naive datetime (2023-07-08 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


{'ASSIGNED': 55, 'AUTOCLOSED': 586, 'COMPLETED': 9}


In [39]:
from django.db.models import Count
from collections import defaultdict

# Ensure different_jobstatus is a list if it's not already
different_jobstatus = list(different_jobstatus)

data = (Jobneed.objects
            .filter(plandatetime__date__range=('2023-05-28', '2023-07-08'),
                    identifier='TASK',
                    jobstatus__in=different_jobstatus)
            .values('plandatetime__date', 'jobstatus')
            .annotate(count=Count('id')))

result = defaultdict(dict)
for item in data:
    date = item['plandatetime__date']
    result[date][item['jobstatus']] = item['count']

# Converts defaultdict back to dict for final result
result = dict(result)
dq([result])

Output()

In [42]:
from django.db.models import Count
from collections import defaultdict
from django.utils.dateparse import parse_date

def get_job_status_counts():
    different_jobstatus = Jobneed.objects.filter(identifier='TASK').values_list('jobstatus', flat=True).distinct()
    data = (Jobneed.objects
                .filter(plandatetime__date__range=(parse_date('2023-05-28'), parse_date('2023-07-08')),
                        identifier='TASK',
                        jobstatus__in=different_jobstatus)
                .values('plandatetime__date', 'jobstatus')
                .annotate(count=Count('id')))

    result = defaultdict(lambda: {status: 0 for status in different_jobstatus})

    for item in data:
        date = item['plandatetime__date']
        result[date][item['jobstatus']] = item['count']

    return dict(result)


In [43]:
get_job_status_counts()

{datetime.date(2023, 5, 28): {'AUTOCLOSED': 1, 'COMPLETED': 0, 'ASSIGNED': 0},
 datetime.date(2023, 5, 29): {'AUTOCLOSED': 1, 'COMPLETED': 0, 'ASSIGNED': 0},
 datetime.date(2023, 5, 30): {'AUTOCLOSED': 1, 'COMPLETED': 0, 'ASSIGNED': 0},
 datetime.date(2023, 5, 31): {'AUTOCLOSED': 1, 'COMPLETED': 1, 'ASSIGNED': 0},
 datetime.date(2023, 6, 1): {'AUTOCLOSED': 1, 'COMPLETED': 0, 'ASSIGNED': 0},
 datetime.date(2023, 6, 2): {'AUTOCLOSED': 1, 'COMPLETED': 0, 'ASSIGNED': 0},
 datetime.date(2023, 6, 3): {'AUTOCLOSED': 1, 'COMPLETED': 1, 'ASSIGNED': 0},
 datetime.date(2023, 6, 5): {'AUTOCLOSED': 1, 'COMPLETED': 1, 'ASSIGNED': 0},
 datetime.date(2023, 6, 6): {'AUTOCLOSED': 1, 'COMPLETED': 1, 'ASSIGNED': 0},
 datetime.date(2023, 6, 7): {'AUTOCLOSED': 1, 'COMPLETED': 1, 'ASSIGNED': 0},
 datetime.date(2023, 6, 8): {'AUTOCLOSED': 1, 'COMPLETED': 0, 'ASSIGNED': 0},
 datetime.date(2023, 6, 9): {'AUTOCLOSED': 1, 'COMPLETED': 0, 'ASSIGNED': 1},
 datetime.date(2023, 6, 10): {'AUTOCLOSED': 0, 'COMPLETED': 

In [52]:
Jobneed.objects.filter(
identifier ='TASK',
plandatetime__date = parse_date('2023-06-10'),).values("jobstatus")

<QuerySet [{'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, {'jobstatus': 'ASSIGNED'}, '...(remaining elements truncated)...']>

In [13]:
PeopleEventlog.objects.values('geojson')[:10]

<QuerySet [{'geojson': {'endlocation': '', 'startlocation': '6XHP+G85, Dhokali, Thane West, Thane, Maharashtra 400607, India'}}, {'geojson': {'endlocation': '', 'startlocation': ''}}, {'geojson': {'endlocation': '', 'startlocation': ''}}, {'geojson': {'endlocation': '', 'startlocation': ''}}, {'geojson': {'endlocation': '', 'startlocation': ''}}, {'geojson': {'endlocation': '', 'startlocation': ''}}, {'geojson': {'endlocation': '', 'startlocation': ''}}, {'geojson': {'endlocation': '', 'startlocation': ''}}, {'geojson': {'endlocation': '', 'startlocation': ''}}, {'geojson': {'endlocation': '', 'startlocation': ''}}]>

In [7]:
dir(l)

['DoesNotExist',
 'LocationStatus',
 'Meta',
 'MultipleObjectsReturned',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_column_name_clashes',
 '_check_constraints',
 '_check_default_pk',
 '_check_field_name_clashes',
 '_check_fields',
 '_check_id_field',
 '_check_index_together',
 '_check_indexes',
 '_check_local_fields',
 '_check_long_column_names',
 '_check_m2m_through_same_relationship',
 '_check_managers',
 '_check_model',
 '_check_model_name_db_lookup_clashes',
 '_check_ordering',
 '_check_property_name_related_field_accessor_clashes',
 '_check_single_primary_key',
 '_check_swappable',
 '_check_unique_together',
 '_do_ins

In [ ]:
l.asset_set.annotate